In [61]:
import pandas as pd
import numpy as np
import mygene

mg = mygene.MyGeneInfo()

In [62]:
count = 0
mapping = {}
with open('P1-02-TTD_drug_download.txt') as fp:
    Lines = fp.readlines()
    target_id = None
    for line in Lines:
        line_content = line.strip().split("\t")
        if len(line_content) >= 3:
            if line_content[1] == "DRUG__ID":
                if target_id is not None:
                    mapping[target_id] = smiles
                    target_id = np.nan
                    smiles = np.nan
                target_id = line_content[2].strip()
            elif line_content[1] == "DRUGINKE":
                smiles = line_content[2].strip()

drugs_df = pd.DataFrame().from_dict(mapping, orient='index', columns=["DRUGINKE"]).dropna()
drugs_df

,DRUGINKE
D00AAN,MSUMHGMGRZWLMN-WXPZYUJUSA-N
D00AAU,KUJAWCSIKNKXLL-UHFFFAOYSA-N
D00ABO,YYLKKYCXAOBSRM-JXMROGBWSA-N
D00ACC,JLFMYEAXZNPWBK-REWPJTCUSA-N
D00ACL,PLETUCDALFTIQN-UHFFFAOYSA-N
...,...
DZ76PB,AKCDDYGYIGPEKL-JQOQJDEVSA-N
DZ8UP4,JDBSZVDIUIRSDG-DAFODLJHSA-N
DZB84T,STPKWKPURVSAJF-LJEWAXOPSA-N
DZK7E0,PSNKGVAXBSAHCH-UHFFFAOYSA-N


In [63]:
count = 0
mapping = {}
with open('P1-01-TTD_target_download.txt') as fp:
    Lines = fp.readlines()
    target_id = None
    for line in Lines:
        line_content = line.strip().split("\t")
        if len(line_content) >= 3:
            if line_content[1] == "TARGETID":
                target_id = line_content[2].strip()

            elif line_content[1] == "GENENAME":
                genename = line_content[2].strip()
                mapping[target_id] = genename
                target_id = None
                genename = None

targets_df = pd.DataFrame().from_dict(mapping, orient='index', columns=["GENE"])
targets_df['GENE'] = targets_df['GENE'].str.strip()
targets_df

,GENE
T47101,FGFR1
T59328,EGFR
T89515,PDF
T08391,JAK2
T07663,PDE5A
...,...
T47438,BACE2; PSENEN; APH1A; APH1B
T83320,SLC12A7
T07601,CAMLG; CAML
T47583,WRN


In [64]:
target_drug_df = pd.read_csv('P1-09-Target_compound_activity.txt', sep="\t")
target_drug_df = target_drug_df.groupby('TTD Drug/Compound ID')['TTD Target ID'].apply(lambda x: ", ".join(x))
target_drug_df

TTD Drug/Compound ID
C01ABG    T55959, T27812, T21945
C01ACX                    T85467
C01ADI            T37693, T76685
C01AFD    T27812, T11448, T01777
C01AGL                    T20401
                   ...          
D0ZY7X    T37848, T57392, T19244
D0ZY8V                    T64765
D0ZZ2L                    T57392
D0ZZ4S                    T32060
D0ZZ4Z    T31204, T83813, T32060
Name: TTD Target ID, Length: 427262, dtype: object

In [65]:
ttd_df = pd.concat([drugs_df, target_drug_df], axis=1, join='inner').set_index('DRUGINKE')
ttd_df

,TTD Target ID
DRUGINKE,
MSUMHGMGRZWLMN-WXPZYUJUSA-N,T70977
YYLKKYCXAOBSRM-JXMROGBWSA-N,T74312
JLFMYEAXZNPWBK-REWPJTCUSA-N,"T52921, T47768, T60693, T58992"
DNTSIBUQMRRYIU-UHFFFAOYSA-N,T58921
DCDHWTNKKCKJLQ-UHFFFAOYSA-N,"T37848, T57392, T19244"
...,...
GDKCAAWSNQLXNX-UHFFFAOYSA-N,"T37848, T57392, T19244"
OFZZAFXTRYWAHX-UHFFFAOYSA-N,T64765
BTFWDABQNDILAN-UHFFFAOYSA-N,T57392


In [66]:
def get_target(x: str):
    targets = str(x).split(",")

    if len(targets) == 1:
        try:
            target = targets_df.loc[x.strip()].values[0].strip()
            return target
        except KeyError:
            return np.nan
    else:
        r = []
        for target in targets:
            try:
                r.append(targets_df.loc[target.strip()].values[0].strip())
            except KeyError:
                pass
        return ", ".join(r)


ttd_df['TTD Target'] = ttd_df['TTD Target ID'].apply(lambda x: get_target(str(x)))
ttd_df = ttd_df.dropna()
ttd_df = ttd_df.drop('TTD Target ID', axis=1).drop_duplicates()
ttd_df

,TTD Target
DRUGINKE,
MSUMHGMGRZWLMN-WXPZYUJUSA-N,GSK3B
YYLKKYCXAOBSRM-JXMROGBWSA-N,FLT3
JLFMYEAXZNPWBK-REWPJTCUSA-N,"OPRL1, OPRM1, OPRK1, OPRD1"
DNTSIBUQMRRYIU-UHFFFAOYSA-N,PPARG
DCDHWTNKKCKJLQ-UHFFFAOYSA-N,"CYP3A4, CYP2D6, CYP2C9"
...,...
GDLIGKIOYRNHDA-UHFFFAOYSA-N,"HTR6, ABCC4, Trypano TPR, SLC6A3, ABCC2, SLC6A..."
NCDNCNXCDXHOMX-XGKFQTDJSA-N,"ABCC4, SLCO1B3, SLC47A2, CYP3A4, Malaria PLA5,..."
WIVGIKIKQHUFOD-UHFFFAOYSA-N,"GRM2, GRM1, GRM8, GRM5"


In [67]:
ttd_df = ttd_df.reset_index().rename({'DRUGINKE': 'InChIKey'}, axis=1).set_index('InChIKey')
ttd_df

,TTD Target
InChIKey,
MSUMHGMGRZWLMN-WXPZYUJUSA-N,GSK3B
YYLKKYCXAOBSRM-JXMROGBWSA-N,FLT3
JLFMYEAXZNPWBK-REWPJTCUSA-N,"OPRL1, OPRM1, OPRK1, OPRD1"
DNTSIBUQMRRYIU-UHFFFAOYSA-N,PPARG
DCDHWTNKKCKJLQ-UHFFFAOYSA-N,"CYP3A4, CYP2D6, CYP2C9"
...,...
GDLIGKIOYRNHDA-UHFFFAOYSA-N,"HTR6, ABCC4, Trypano TPR, SLC6A3, ABCC2, SLC6A..."
NCDNCNXCDXHOMX-XGKFQTDJSA-N,"ABCC4, SLCO1B3, SLC47A2, CYP3A4, Malaria PLA5,..."
WIVGIKIKQHUFOD-UHFFFAOYSA-N,"GRM2, GRM1, GRM8, GRM5"


In [68]:
repurposing_hub_drug_targets = pd.read_csv("../../data/primary_data/repurposing_hub_drug_targets.txt", sep="\t").drop(
    ['MOA', 'Phase'], axis=1).set_index('Name')
repurposing_hub_drug_targets = repurposing_hub_drug_targets[
    ~repurposing_hub_drug_targets.index.duplicated(keep='first')]
repurposing_hub_drug_targets

,Target
Name,
(R)-(-)-apomorphine,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3..."
(R)-(-)-rolipram,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A"
(R)-baclofen,"GABBR1, GABBR2"
(S)-(+)-rolipram,"PDE4B, PDE4D"
"[sar9,met(o2)11]-substance-p",TACR1
...,...
8-M-PDOT,"MTNR1A, MTNR1B"
80841-78-7,NaN
9-aminoacridine,NaN


In [69]:
repurposing_hub_drugs = pd.read_csv('../../data/primary_data/repurposing_hub_drugs.txt', skiprows=9, sep='\t')
repurposing_hub_drugs['broad_id'] = repurposing_hub_drugs['broad_id'].apply(lambda x: x[:-9])
repurposing_hub_drugs = repurposing_hub_drugs.loc[:, ['broad_id', 'pert_iname', 'InChIKey']].drop_duplicates().rename(
    {'pert_iname': 'Name'}, axis=1).set_index('Name')
repurposing_hub_drugs = repurposing_hub_drugs[~repurposing_hub_drugs.index.duplicated(keep='first')]
repurposing_hub_drugs

,broad_id,InChIKey
Name,,
(R)-(-)-apomorphine,BRD-K76022557,VMWNQDUVQKEIOC-CYBMUJFWSA-N
(R)-(-)-rolipram,BRD-K75516118,HJORMJIFDVBMOB-LBPRGKRZSA-N
(R)-baclofen,BRD-K62353271,KPYSYYIEGFHWSV-QMMMGPOBSA-N
(S)-(+)-rolipram,BRD-K65856711,HJORMJIFDVBMOB-GFCCVEGCSA-N
"[sar9,met(o2)11]-substance-p",BRD-K89787693,OUPXSLGGCPUZJJ-SARDKLJWSA-N
...,...,...
zoxazolamine,BRD-K66353228,YGCODSQDUUUKIV-UHFFFAOYSA-N
ZSET1446,BRD-K35984734,QZWYXEBIQWJXAR-UHFFFAOYSA-N
ZSTK-474,BRD-K63068307,HGVNLRPZOWWDKD-UHFFFAOYSA-N


In [70]:
drh = pd.concat([repurposing_hub_drug_targets, repurposing_hub_drugs], axis=1, join='inner').set_index('InChIKey')
drh

,Target,broad_id
InChIKey,,
VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...",BRD-K76022557
HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A",BRD-K75516118
KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",BRD-K62353271
HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D",BRD-K65856711
OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,BRD-K89787693
...,...,...
RVIGBTUDFAGRTQ-NSHDSACASA-N,"MTNR1A, MTNR1B",BRD-A07232941
QCLFSYYUWPUWQR-UHFFFAOYSA-N,NaN,BRD-K03044000
XJGFWWJLMVZSIG-UHFFFAOYSA-N,NaN,BRD-K00535541


In [71]:
drh = drh[~drh.index.duplicated(keep='first')]
drh

,Target,broad_id
InChIKey,,
VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...",BRD-K76022557
HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A",BRD-K75516118
KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",BRD-K62353271
HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D",BRD-K65856711
OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,BRD-K89787693
...,...,...
RVIGBTUDFAGRTQ-NSHDSACASA-N,"MTNR1A, MTNR1B",BRD-A07232941
QCLFSYYUWPUWQR-UHFFFAOYSA-N,NaN,BRD-K03044000
XJGFWWJLMVZSIG-UHFFFAOYSA-N,NaN,BRD-K00535541


In [72]:
ttd_df = ttd_df[~ttd_df.index.duplicated(keep='first')]
ttd_df

,TTD Target
InChIKey,
MSUMHGMGRZWLMN-WXPZYUJUSA-N,GSK3B
YYLKKYCXAOBSRM-JXMROGBWSA-N,FLT3
JLFMYEAXZNPWBK-REWPJTCUSA-N,"OPRL1, OPRM1, OPRK1, OPRD1"
DNTSIBUQMRRYIU-UHFFFAOYSA-N,PPARG
DCDHWTNKKCKJLQ-UHFFFAOYSA-N,"CYP3A4, CYP2D6, CYP2C9"
...,...
GDLIGKIOYRNHDA-UHFFFAOYSA-N,"HTR6, ABCC4, Trypano TPR, SLC6A3, ABCC2, SLC6A..."
NCDNCNXCDXHOMX-XGKFQTDJSA-N,"ABCC4, SLCO1B3, SLC47A2, CYP3A4, Malaria PLA5,..."
WIVGIKIKQHUFOD-UHFFFAOYSA-N,"GRM2, GRM1, GRM8, GRM5"


In [73]:
ttd_drh_merge = pd.concat([drh, ttd_df], axis=1, join='outer')
ttd_drh_merge

,Target,broad_id,TTD Target
InChIKey,,,
VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...",BRD-K76022557,"HTR1A, ABCC4, DRD3, DRD2, DRD4, ABCC2, ABCB11,..."
HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A",BRD-K75516118,"PDE4A, PDE4C, PDE4D, PDE4B"
KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",BRD-K62353271,NaN
HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D",BRD-K65856711,"PDE4A, PDE4D"
OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,BRD-K89787693,NaN
...,...,...,...
GAYSOZKZPOVDSB-HZMBPMFUSA-N,NaN,NaN,"ADRB1, DRD2, HTR3A, HTR4, ADRB2, DRD1"
UXNUQVZRIJPJBL-UHFFFAOYSA-N,NaN,NaN,"MKNK2, MKNK1, BCR-ABL1"
LBWQSAZEYIZZCE-SNVBAGLBSA-N,NaN,NaN,"CRHR2, CRHBP, CRHR1, TACR2"


In [74]:
ttd_drh_merge.dropna(subset='broad_id', inplace=True)
ttd_drh_merge = ttd_drh_merge.reset_index().set_index('broad_id')

In [75]:
ttd_drh_merge.to_csv('ttd_drh_merge.csv', index=True)

In [76]:
ttd_drh_merge

,InChIKey,Target,TTD Target
broad_id,,,
BRD-K76022557,VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...","HTR1A, ABCC4, DRD3, DRD2, DRD4, ABCC2, ABCB11,..."
BRD-K75516118,HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A","PDE4A, PDE4C, PDE4D, PDE4B"
BRD-K62353271,KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",NaN
BRD-K65856711,HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D","PDE4A, PDE4D"
BRD-K89787693,OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,NaN
...,...,...,...
BRD-A07232941,RVIGBTUDFAGRTQ-NSHDSACASA-N,"MTNR1A, MTNR1B",NaN
BRD-K03044000,QCLFSYYUWPUWQR-UHFFFAOYSA-N,NaN,NaN
BRD-K00535541,XJGFWWJLMVZSIG-UHFFFAOYSA-N,NaN,NaN
